<a href="https://colab.research.google.com/github/Chancelor2023/foodrecognitionknu/blob/main/Japanese_food_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

In [ ]:
# copy kaggle.json to /root/.kaggle/ folder so that kaggle cli can access it.
!mkdir /.kaggle
!mv kaggle.json /.kaggle
!mv /.kaggle /root/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d rkuo2000/uecfood256

In [ ]:
import zipfile
with zipfile.ZipFile("uecfood256.zip", 'r') as zip_ref:
    zip_ref.extractall("food-256")

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

In [ ]:
import os
import random
from shutil import copy2
from PIL import Image

# 1. Load categories from category.txt
category_file = "food-256/UECFOOD256/category.txt"
with open(category_file, 'r', encoding='utf-8') as f:
    classes = [line.strip().split()[1] for line in f.readlines()[1:]]  # Skip the first line

# 2. Define the ratios
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

# 3. Create folder structure
base_path = "food-256"
splits = ["train", "val", "test"]
for split in splits:
    os.makedirs(os.path.join(base_path, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(base_path, split, "labels"), exist_ok=True)

# 4. Loop through category directories
uec_path = os.path.join(base_path, "UECFOOD256")
for category_dir in os.listdir(uec_path):
    category_path = os.path.join(uec_path, category_dir)
    if not os.path.isdir(category_path) or not category_dir.isdigit():
        continue

    bb_info_file = os.path.join(category_path, "bb_info.txt")
    if not os.path.exists(bb_info_file):
        continue

    # Load images
    images = [img for img in os.listdir(category_path) if img.endswith(".jpg")]
    random.shuffle(images)  # Shuffle the images

    # Split the images
    split_train = int(len(images) * TRAIN_RATIO)
    split_val = int(len(images) * (TRAIN_RATIO + VAL_RATIO))

    train_images = images[:split_train]
    val_images = images[split_train:split_val]
    test_images = images[split_val:]

    # Loop through each image
    for image_name in images:
        # Define source and destination paths
        image_path = os.path.join(category_path, image_name)
        split = (
            "train" if image_name in train_images else
            "val" if image_name in val_images else
            "test"
        )
        dest_image_dir = os.path.join(base_path, split, "images")
        dest_label_dir = os.path.join(base_path, split, "labels")
        dest_image_path = os.path.join(dest_image_dir, image_name)
        dest_label_path = os.path.join(dest_label_dir, image_name.replace(".jpg", ".txt"))

        # Load the image dimensions
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        # Extract and convert annotations
        with open(bb_info_file, 'r') as f_in, open(dest_label_path, 'w') as f_out:
            for line in f_in:
                parts = line.strip().split()
                if len(parts) < 5 or parts[0] != image_name[:-4]:
                    continue

                x1, y1, x2, y2 = map(int, parts[1:5])
                class_id = int(category_dir) - 1  # YOLO class IDs start at 0
                x_center = (x1 + x2) / (2 * image_width)
                y_center = (y1 + y2) / (2 * image_height)
                width = (x2 - x1) / image_width
                height = (y2 - y1) / image_height

                # Write YOLO annotations
                f_out.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        # Copy the image to the corresponding folder
        copy2(image_path, dest_image_path)

print("Conversion completed. The data is organized.")


In [ ]:
import os

# Image folder paths
base_path = "food-256"
train_path = os.path.join(base_path, "train", "images")
val_path = os.path.join(base_path, "val", "images")
test_path = os.path.join(base_path, "test", "images")

# Number of classes
nc = 256  # 256 classes in your dataset

# Load class names from category.txt
category_file = "food-256/UECFOOD256/category.txt"
with open(category_file, 'r', encoding='utf-8') as f:
    classes = [line.strip().split()[1] for line in f.readlines()[1:]]  # Skip the first line (header)

# Create the data.yaml file
yaml_content = f"""
train: {os.path.abspath(train_path)}
val: {os.path.abspath(val_path)}
test: {os.path.abspath(test_path)}  # Remove this line if you don't have a test set

nc: {nc}

names:
"""

for i, class_name in enumerate(classes):
    yaml_content += f"  {i}: '{class_name}'\n"

# Write the data.yaml file
with open(os.path.join(base_path, "data.yaml"), 'w') as f:
    f.write(yaml_content)

print("The data.yaml file has been successfully generated.")


In [ ]:
import os
import random
import shutil

# Original folder and reduced folder
original_dataset = "/content/food-256"
reduced_dataset = "/content/food-256-reduced"

# Create folder structure for the reduced dataset
os.makedirs(reduced_dataset, exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "train/images"), exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "train/labels"), exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "val/images"), exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "val/labels"), exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "test/images"), exist_ok=True)
os.makedirs(os.path.join(reduced_dataset, "test/labels"), exist_ok=True)

# Reduction ratio (here, 25% of the images)
reduction_ratio = 0.25

# Loop through the train, val, test subfolders
for split in ["train", "val", "test"]:
    images_dir = os.path.join(original_dataset, split, "images")
    labels_dir = os.path.join(original_dataset, split, "labels")

    reduced_images_dir = os.path.join(reduced_dataset, split, "images")
    reduced_labels_dir = os.path.join(reduced_dataset, split, "labels")

    # List all images
    images = os.listdir(images_dir)
    num_images = len(images)

    # Randomly select 25% of the images
    selected_images = random.sample(images, int(num_images * reduction_ratio))

    # Copy the images and their annotations
    for image in selected_images:
        image_path = os.path.join(images_dir, image)
        label_path = os.path.join(labels_dir, image.replace(".jpg", ".txt"))

        shutil.copy(image_path, os.path.join(reduced_images_dir, image))
        shutil.copy(label_path, os.path.join(reduced_labels_dir, image.replace(".jpg", ".txt")))

print(f"Reduced dataset successfully created in {reduced_dataset}")


In [ ]:
import os

# Image folder paths
base_path = "food-256-reduced"
train_path = os.path.join(base_path, "train", "images")
val_path = os.path.join(base_path, "val", "images")
test_path = os.path.join(base_path, "test", "images")

# Number of classes
nc = 256  # 256 classes in your dataset

# Load class names from category.txt
category_file = "food-256/UECFOOD256/category.txt"
with open(category_file, 'r', encoding='utf-8') as f:
    classes = [line.strip().split()[1] for line in f.readlines()[1:]]  # Skip the first line (header)

# Create the data.yaml file
yaml_content = f"""
train: {os.path.abspath(train_path)}
val: {os.path.abspath(val_path)}
test: {os.path.abspath(test_path)}  # Remove this line if you don't have a test set

nc: {nc}

names:
"""

for i, class_name in enumerate(classes):
    yaml_content += f"  {i}: '{class_name}'\n"

# Write the data.yaml file
with open(os.path.join(base_path, "data.yaml"), 'w') as f:
    f.write(yaml_content)

print("The data.yaml file has been successfully generated.")


In [ ]:
!python yolov5/train.py --img 640 --batch 16 --epochs 50 --data /content/food-256-reduced/data.yaml --weights yolov5s.pt


In [ ]:
python val.py --weights runs/train/exp/weights/best.pt --data data.yaml


In [ ]:
python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source chemin/vers/image.jpg


In [ ]:
import shutil

# Path to the folder you want to delete
folder_path = "/content/reduced-food-256"

# Remove the folder and all its contents
shutil.rmtree(folder_path)
print(f"Folder {folder_path} has been deleted.")
